# DSCI 617 – Homework 04
**Felix Asare**

In [0]:
# Libraries (SparkSession, col and expr)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder.getOrCreate()


## Load Diamond Data

In [0]:
# Creating a schema for the diamonds data
diamonds = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .schema(
        'carat DOUBLE, cut STRING, color STRING, clarity STRING, depth DOUBLE, '
        'table DOUBLE, price INTEGER, x DOUBLE, y DOUBLE, z DOUBLE'
    )
    .csv('/FileStore/tables/diamonds.txt')
)

diamonds.printSchema()

diamonds.createOrReplaceTempView('diamonds')


root
 |-- carat: double (nullable = true)
 |-- cut: string (nullable = true)
 |-- color: string (nullable = true)
 |-- clarity: string (nullable = true)
 |-- depth: double (nullable = true)
 |-- table: double (nullable = true)
 |-- price: integer (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



## Problem 1: Grouping By Cut

In [0]:
from pyspark.sql.types import IntegerType
# Create a rank_cut function
def rank_cut(cut):
    rank_dict = {
        'Fair': 1,
        'Good': 2,
        'Very Good': 3,
        'Premium': 4,
        'Ideal': 5
    }
    return rank_dict.get(cut, None)
    
# register the function as a UDF
rank_cut_udf = udf(rank_cut)


In [0]:
from pyspark.sql.functions import udf
# Group diamonds by cut
grouped_cut = spark.sql("""
    SELECT cut, 
           COUNT(*) AS n_diamonds, 
           ROUND(AVG(price), 0) AS avg_price, 
           ROUND(AVG(carat), 2) AS avg_carat, 
           ROUND(AVG(depth), 2) AS avg_depth, 
           ROUND(AVG(table), 2) AS avg_table
    FROM diamonds
    GROUP BY cut
""").withColumn("cut_rank", rank_cut_udf(col("cut"))).orderBy("cut_rank").show()



+---------+----------+---------+---------+---------+---------+--------+
|      cut|n_diamonds|avg_price|avg_carat|avg_depth|avg_table|cut_rank|
+---------+----------+---------+---------+---------+---------+--------+
|     Fair|      1610|   4359.0|     1.05|    64.04|    59.05|       1|
|     Good|      4906|   3929.0|     0.85|    62.37|    58.69|       2|
|Very Good|     12082|   3982.0|     0.81|    61.82|    57.96|       3|
|  Premium|     13791|   4584.0|     0.89|    61.26|    58.75|       4|
|    Ideal|     21551|   3458.0|      0.7|    61.71|    55.95|       5|
+---------+----------+---------+---------+---------+---------+--------+



## Problem 2: Filtering based on Carat Size

In [0]:
# Use a loop to count the number of diamonds
bins = ['[0,1)', '[1,2)', '[2,3)', '[3,4)', '[4,5)', '[5,6)']
for bin_range in bins:
    lower, upper = bin_range.strip('[]').replace(')', '').split(',')
    count = diamonds.filter((diamonds.carat >= lower) & (diamonds.carat < upper)).count()
    print(f"The Number of diamonds in the carat range [{lower}, {upper}) is {count}")

The Number of diamonds in the carat range [0, 1) is 34880
The Number of diamonds in the carat range [1, 2) is 16906
The Number of diamonds in the carat range [2, 3) is 2114
The Number of diamonds in the carat range [3, 4) is 34
The Number of diamonds in the carat range [4, 5) is 5
The Number of diamonds in the carat range [5, 6) is 1


## Problem 3: Binning by Carat Size

In [0]:
# Function to bin carat
def carat_bin(carat):
    if 0 <= carat < 1:
        return '[0, 1)'
    elif 1 <= carat < 2:
        return '[1, 2)'
    elif 2 <= carat < 3:
        return '[2, 3)'
    elif 3 <= carat < 4:
        return '[3, 4)'
    elif 4 <= carat < 5:
        return '[4, 5)'
    elif 5 <= carat < 6:
        return '[5, 6)'
    else:
        return None 

# Register the function as a UDF
carat_bin_udf = udf(carat_bin)

In [0]:
# Add a new column 'carat_bin' to the DataFrame
diamonds.withColumn('carat_bin', carat_bin_udf(col('carat'))).show()


+-----+---------+-----+-------+-----+-----+-----+----+----+----+---------+
|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|carat_bin|
+-----+---------+-----+-------+-----+-----+-----+----+----+----+---------+
| 0.23|    Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|   [0, 1)|
| 0.21|  Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|   [0, 1)|
| 0.23|     Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|   [0, 1)|
| 0.29|  Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|   [0, 1)|
| 0.31|     Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|   [0, 1)|
| 0.24|Very Good|    J|   VVS2| 62.8| 57.0|  336|3.94|3.96|2.48|   [0, 1)|
| 0.24|Very Good|    I|   VVS1| 62.3| 57.0|  336|3.95|3.98|2.47|   [0, 1)|
| 0.26|Very Good|    H|    SI1| 61.9| 55.0|  337|4.07|4.11|2.53|   [0, 1)|
| 0.22|     Fair|    E|    VS2| 65.1| 61.0|  337|3.87|3.78|2.49|   [0, 1)|
| 0.23|Very Good|    H|    VS1| 59.4| 61.0|  338| 4.0|4.05|2.39|   [0, 1)|
|  0.3|     Good|    J|  

## Load IMDB Data

In [0]:
# Load movies data
movies = spark.read.option("header", True).option("sep", "\t").csv('/FileStore/tables/imdb/movies.txt')


movies.printSchema()

movies.createOrReplaceTempView('movies')

root
 |-- imdb_title_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- country: string (nullable = true)
 |-- language: string (nullable = true)



In [0]:
# Load names data
names = spark.read.option("header", True).option("sep", "\t").csv('/FileStore/tables/imdb/names.txt')

names.printSchema()
names.createOrReplaceTempView('names')

root
 |-- imdb_name_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_name: string (nullable = true)
 |-- height: string (nullable = true)
 |-- bio: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- date_of_death: string (nullable = true)



In [0]:
# Load title_principals data
title_principals = spark.read.option("header", True).option("sep", "\t").csv('/FileStore/tables/imdb/title_principals-1.txt')

title_principals.printSchema()
title_principals.createOrReplaceTempView('title_principals')

root
 |-- imdb_title_id: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- imdb_name_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- characters: string (nullable = true)



In [0]:
# Load ratings data
ratings = spark.read.option("header", True).option("sep", "\t").csv('/FileStore/tables/imdb/ratings-1.txt')

ratings.printSchema()
ratings.createOrReplaceTempView('ratings')

root
 |-- imdb_title_id: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- total_votes: string (nullable = true)



In [0]:
# Number os records in movies, names, ratings, title_principals
print(f"Number of records in movies: {movies.count()}")
print(f"Number of records in names: {names.count()}")
print(f"Number of records in title_principals: {title_principals.count()}")
print(f"Number of records in ratings: {ratings.count()}")


Number of records in movies: 85855
Number of records in names: 297710
Number of records in title_principals: 835513
Number of records in ratings: 85855


## Problem 4: Number of Appearances by Actor

In [0]:
from pyspark.sql import functions as F
result = (
    title_principals
    .filter((col("category") == "actor") | (col("category") == "actress"))
    .groupBy("imdb_name_id")
    .agg(F.count("*").alias("appearances"))
    .join(names, "imdb_name_id", "inner")
    .select("name", "appearances") 
    .orderBy(col("appearances").desc()) 
    .limit(16)
)
result.show()

+-----------------+-----------+
|             name|appearances|
+-----------------+-----------+
|         Mohanlal|        163|
| Amitabh Bachchan|        142|
|        Mammootty|        140|
|     Eric Roberts|        133|
|       John Wayne|        132|
| Gérard Depardieu|        130|
|      Prakash Raj|        125|
|     Akshay Kumar|        115|
|   Michael Madsen|        107|
|         Andy Lau|        102|
|Catherine Deneuve|        101|
|      Anupam Kher|         99|
|     Brahmanandam|         99|
|    Michael Caine|         94|
|       Ajay Devgn|         94|
|  Christopher Lee|         93|
+-----------------+-----------+



## Problem 5: Average Rating by Director

In [0]:
# #Average director rating
from pyspark.sql.functions import col, count, sum, avg, round
avg_result = (
    title_principals
    .filter(col('category') == 'director')
    .join(ratings, 'imdb_title_id', 'inner')
    .groupBy('imdb_name_id')
    .agg(
        count('*').alias('num_films'),
        sum('total_votes').alias('total_votes'),
        round(avg('rating'), 2).alias('avg_rating')
    )
    .filter(col('total_votes') >= 1000000)
    .filter(col('num_films') >= 5)
    .join(names, 'imdb_name_id', 'inner')
    .select('name', 'num_films', 'total_votes', 'avg_rating')
    .orderBy(col('avg_rating').desc())
    .limit(16)
)

avg_result.show(truncate=False)

+---------------------+---------+-----------+----------+
|name                 |num_films|total_votes|avg_rating|
+---------------------+---------+-----------+----------+
|Christopher Nolan    |11       |1.1653144E7|8.22      |
|Lee Unkrich          |5        |3329612.0  |8.14      |
|Hayao Miyazaki       |12       |2254496.0  |8.01      |
|Quentin Tarantino    |14       |9460772.0  |7.93      |
|Sergio Leone         |7        |1720654.0  |7.93      |
|Stanley Kubrick      |13       |4232356.0  |7.78      |
|David Fincher        |10       |6944421.0  |7.76      |
|Sam Mendes           |10       |3067512.0  |7.73      |
|Alejandro G. Iñárritu|7        |2067540.0  |7.61      |
|Wes Anderson         |9        |2173090.0  |7.61      |
|Peter Jackson        |13       |7304418.0  |7.58      |
|Brad Bird            |6        |2294748.0  |7.57      |
|Alfonso Cuarón       |8        |2078975.0  |7.54      |
|Andrew Stanton       |5        |2649551.0  |7.52      |
|Bong Joon Ho         |8       

## Problem 6: Actors Appearing in Horror Films

In [0]:
# Horror films
horror_films = movies.filter(expr('genre LIKE "%Horror%"'))
horror_films.count()

9557

In [0]:
horror_result = (
    title_principals
    .filter((col('category') == 'actor') | (col('category') == 'actress'))
    .join(horror_films, 'imdb_title_id', 'inner')
    .groupBy('imdb_name_id')
    .agg(count('imdb_title_id').alias('num_films'))
    .join(names, 'imdb_name_id', 'inner')
    .select('name', 'num_films')
    .orderBy(col('num_films').desc())
    .limit(16)
)

# Display results
horror_result.show()

+----------------+---------+
|            name|num_films|
+----------------+---------+
| Christopher Lee|       56|
|   Peter Cushing|       47|
|   Boris Karloff|       46|
|  John Carradine|       43|
|     Bela Lugosi|       38|
|   Vincent Price|       34|
| Lance Henriksen|       33|
|    Eric Roberts|       29|
|  Lon Chaney Jr.|       28|
|    Bill Moseley|       27|
|       Tony Todd|       27|
|     Paul Naschy|       26|
|Donald Pleasence|       26|
|       Sergey A.|       23|
|  Robert Englund|       23|
|     Brad Dourif|       23|
+----------------+---------+

